In [11]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets.utils import download_and_extract_archive
from PIL import Image

from pytorch_ood.detector import EnergyBased
from pytorch_ood.utils import OODMetrics
from pytorch_ood.model import WideResNet
from pytorch_ood.dataset.img import Textures

root = "./data/textures"
preprocess = WideResNet.transform_for("er-cifar10-tune")
dataset_train = Textures(root=root, download=True, transform=preprocess)
dataset_in_test = Textures(root=root, download=True, transform=preprocess)
dataset_out_test = Textures(root=root, download=True, transform=preprocess)
# data_loader = DataLoader(textures_dataset, batch_size=64, shuffle=False, num_workers=4)

TypeError: Textures.__init__() got an unexpected keyword argument 'train'

In [10]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

from pytorch_ood.dataset.img import Textures
from pytorch_ood.detector import OpenMax
from pytorch_ood.model import WideResNet
from pytorch_ood.utils import OODMetrics, ToUnknown, fix_random_seed

fix_random_seed(123)

device = "cuda:0"
trans = WideResNet.transform_for("cifar10-pt")

dataset_train = CIFAR10(root="data", train=True, download=True, transform=trans)
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
dataset_out_test = Textures(
root="data", download=True, transform=trans, target_transform=ToUnknown()
)

train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True)

# create data loaders
test_loader = DataLoader(dataset_in_test + dataset_out_test, batch_size=128)

model = WideResNet(num_classes=10, pretrained="cifar10-pt").to(device).eval()
detector = OpenMax(model, tailsize=25, alpha=5, euclid_weight=0.5)
detector.fit(train_loader, device=device)
metrics = OODMetrics()

for x, y in test_loader:
    metrics.update(detector(x.to(device)), y)

print(metrics.compute())

ValueError: Data must contain ID and OOD samples.